<a href="https://colab.research.google.com/github/Dp170325/Ibm_aicte_cycle-3-aug_2025-internship/blob/main/ev_ibm_internshhip_aicte_aug2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from google.colab import files
uploaded = files.upload()  # Select the ZIP file you downloaded

Saving archive (1).zip to archive (1) (1).zip


In [7]:
import zipfile
import os

with zipfile.ZipFile("/content/archive (1).zip", 'r') as zip_ref:
    zip_ref.extractall()


In [37]:
# Step 1: Import libraries   =

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




In [36]:
# Step 2: Load the dataset
df = pd.read_csv("/content/ev_charging_patterns.csv")


,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [39]:
#    Step 3: Inspect the dataset
print("Shape:", df.shape)
print("Columns:", df.columns)
print(df.head())
print(df.info())

Shape: (1320, 20)
Columns: Index(['User ID', 'Vehicle Model', 'Battery Capacity (kWh)',
       'Charging Station ID', 'Charging Station Location',
       'Charging Start Time', 'Charging End Time', 'Energy Consumed (kWh)',
       'Charging Duration (hours)', 'Charging Rate (kW)',
       'Charging Cost (USD)', 'Time of Day', 'Day of Week',
       'State of Charge (Start %)', 'State of Charge (End %)',
       'Distance Driven (since last charge) (km)', 'Temperature (°C)',
       'Vehicle Age (years)', 'Charger Type', 'User Type'],
      dtype='object')
  User ID Vehicle Model  Battery Capacity (kWh) Charging Station ID  \
0  User_1        BMW i3              108.463007         Station_391   
1  User_2  Hyundai Kona              100.000000         Station_428   
2  User_3    Chevy Bolt               75.000000         Station_181   
3  User_4  Hyundai Kona               50.000000         Station_327   
4  User_5  Hyundai Kona               50.000000         Station_108   

  Charging Stati

In [43]:
# Finding Unique values by column

df.nunique()

,0
User ID,1254
Vehicle Model,5
Battery Capacity (kWh),137
Charging Station ID,457
Charging Station Location,5
Charging Start Time,1254
Charging End Time,1244
Energy Consumed (kWh),1254
Charging Duration (hours),1254
Charging Rate (kW),1191


In [79]:
# Average Charging Duration by user

avg_duration = df.groupby('User Type')['Charging Duration (hours)'].mean()
avg_duration

,Charging Duration (hours)
User Type,
Casual Driver,2.289826
Commuter,2.320847
Long-Distance Traveler,2.222083


In [66]:
# Step 4: Convert timestamps
df['start_time'] = pd.to_datetime(df['Charging Start Time'])
df['end_time'] = pd.to_datetime(df['Charging End Time'])

In [67]:
#       Step 5: Feature Engineering
df['duration_hours'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 3600
df['hour_of_day'] = df['start_time'].dt.hour
df['day_of_week'] = df['start_time'].dt.day_name()

def time_bucket(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'
df['time_of_day'] = df['hour_of_day'].apply(time_bucket)

In [71]:
#  Step 6: Handle missing values
# Drop rows missing
df = df.dropna(subset=['Energy Consumed (kWh)', 'Battery Capacity (kWh)'])

# Fill missing temp with median (optional)
if 'Temperature (°C)' in df.columns:
    df['Temperature (°C)'] = df['Temperature (°C)'].fillna(df['Temperature (°C)'].median())
else:
    df['Temperature (°C)'] = 25

In [86]:
#Encode categorical variables
df = pd.get_dummies(df, columns=['time_of_day', 'day_of_week'], drop_first=True)

In [87]:
# Step 8: Select features
base_features = [
    'Battery Capacity (kWh)', 'duration_hours', 'Vehicle Age (years)', 'Temperature (°C)', 'hour_of_day'
]

# Add one-hot encoded features
encoded_features = [col for col in df.columns if
                    col.startswith('Vehicle Model_') or
                    col.startswith('Charger Type_') or
                    col.startswith('time_of_day_') or
                    col.startswith('day_of_week_')]

features = base_features + encoded_features

In [77]:
#  Step 9: Define X and y
X = df[features]
y = df['Energy Consumed (kWh)']



In [104]:
# Step 10: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [103]:
print(" Data is ready for modeling (Week 2)")

 Data is ready for modeling (Week 2)


In [105]:
# STEP 1: Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error



In [ ]:
# STEP 2: Basic cleaning (drop rows with missing important data)
df = df.dropna(subset=['Charging Start Time', 'Charging End Time', 'Battery Capacity (kWh)', 'Energy Consumed (kWh)'])



In [ ]:
# STEP 4: Convert time columns to datetime
df['start_time'] = pd.to_datetime(df['Charging Start Time'])
df['end_time'] = pd.to_datetime(df['Charging End Time'])



In [108]:
# STEP 5: Create duration column in hours
df['duration_hours'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 3600



In [109]:
# STEP 6: Add a few simple features
df['hour'] = df['start_time'].dt.hour
df['day'] = df['start_time'].dt.dayofweek  # 0 = Monday, 6 = Sunday



In [110]:
# STEP 7: Select features and target
X = df[['Battery Capacity (kWh)', 'duration_hours', 'hour', 'day']]
y = df['Energy Consumed (kWh)']




In [111]:
# STEP 8: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [112]:
# STEP 9: Train a simple model
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [113]:
# STEP 10: Predict and evaluate
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)


print(" Mean Absolute Error (MAE):", round(mae, 2))

 Mean Absolute Error (MAE): 18.72


In [114]:
print("Model trained successfully!")

Model trained successfully!


In [115]:
print(" Data is ready for modeling (Week 2)")

 Data is ready for modeling (Week 2)


In [92]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.4 MB/s eta 0:00:00


In [100]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

# Title
st.title("EV Charging Energy Prediction App")

# Sidebar inputs
st.sidebar.header("Input EV Charging Details")

battery = st.sidebar.number_input("Battery Capacity (kWh)", min_value=10.0, max_value=150.0, value=60.0)
duration = st.sidebar.number_input("Charging Duration (hours)", min_value=0.1, max_value=12.0, value=2.0)
hour = st.sidebar.slider("Start Hour (0–23)", min_value=0, max_value=23, value=14)
day = st.sidebar.selectbox("Day of the Week", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
day_dict = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3,
            "Friday": 4, "Saturday": 5, "Sunday": 6}
day_num = day_dict[day]

# Prediction
if st.sidebar.button("Predict Energy Consumed"):
    # Model input
    input_data = pd.DataFrame({
        'Battery Capacity (kWh)': [battery],
        'duration_hours': [duration],
        'hour': [hour],
        'day': [day_num]
    })

    # Load model (use your trained model)
    model = pickle.load(open("ev_model.pkl", "rb"))
    prediction = model.predict(input_data)[0]

    st.success(f"Estimated Energy Consumed: {round(prediction, 2)} kWh")




2025-09-13 21:44:15.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.229 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 21:44:15.232 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar